# Tutorial - Step4 : Molecular translation using finetuned models

## 1. Import requirements

In [1]:
import os
import sys
import pandas as pd
import time
import tqdm
import torch
from torch.utils.data import DataLoader
from rdkit.Chem.rdmolfiles import MolFromSmiles

In [2]:
sys.path = [os.path.abspath(os.path.join(os.getcwd(), os.pardir))] + sys.path

In [3]:
from COMA.dataset import ValidationSmilesDataset
from COMA.vae import SmilesAutoencoder
from COMA.properties import drd2, similarity

## 2. Configure GPU (if available)

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)

cuda:0


## 3. Specify a target property

In [4]:
PROPERTY_NAME = "drd2"
SCORING_FT = drd2

## 4. Set directories (for inputs and outputs)

In [5]:
input_data_dir = os.path.abspath(os.path.join(os.pardir, "DATA", PROPERTY_NAME))
input_ckpt_dir = f"outputs_1-2_{PROPERTY_NAME.upper()}_finetuning"

In [6]:
output_dir = f"outputs_1-4_{PROPERTY_NAME.upper()}_translation"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

## 5. Set file names

In [7]:
filepath_test = os.path.join(input_data_dir, "rdkit_test.txt")

In [8]:
filepath_pretrain_ckpt     = os.path.join(input_ckpt_dir, "checkpoints.pt")
filepath_pretrain_configs  = os.path.join(input_ckpt_dir, "configs.csv")
filepath_pretrain_char2idx = os.path.join(input_ckpt_dir, "char2idx.csv")

In [9]:
filepath_output = os.path.join(output_dir, f"COMA_{PROPERTY_NAME.upper()}.csv")

## 6. Load datasets (for test)

In [10]:
dataset_test = ValidationSmilesDataset(filepath_test, filepath_pretrain_char2idx, device=device)

## 7. Load a pretrained generator of MTMR

In [11]:
## Model configuration
model_configs = {"hidden_size"    :None,
                 "latent_size"    :None,
                 "num_layers"     :None,
                 "vocab_size"     :None,
                 "sos_idx"        :None,
                 "eos_idx"        :None,
                 "pad_idx"        :None,
                 "device"         :device,
                 "filepath_config":filepath_pretrain_configs}

## Model initialization
generator = SmilesAutoencoder(**model_configs)

## Load pretrained model
generator.load_model(filepath_pretrain_ckpt)

## 8. Perform molecular translation on the Test dataset

In [12]:
K = 20 # repetition count of translation

generated = [] # initialize a list of outputs

for batch in tqdm.tqdm(DataLoader(dataset_test, batch_size=1, shuffle=False, drop_last=False, pin_memory=use_cuda)):
    batch_smiles = dataset_test.encode(batch["smiles_s"], batch["length_s"].max())
    batch_length = batch["length_s"]
    ## translation
    for _ in range(K):
        seq = generator.predict(batch_smiles, batch_length)
        smi = dataset_test.decode(seq)[0] # assumption: batch_size=1
        if MolFromSmiles(smi) is not None:
            generated.append((batch["smiles_s"][0][1:-1], smi))
        else:
            generated.append((batch["smiles_s"][0][1:-1], "None"))
        
df_generated = pd.DataFrame.from_records(generated)

100%|██████████| 1000/1000 [06:42<00:00,  2.49it/s]


## 9. Evaluate the translation outputs

In [13]:
scores = []
for smi_src, smi_tar in tqdm.tqdm(df_generated.values):
    if smi_tar == "None": smi_tar=None
    sim2D = similarity(smi_src, smi_tar)
    try:
        scores.append((smi_src, smi_tar, sim2D, SCORING_FT(smi_tar)))
    except Exception as e:
        scores.append((smi_src, smi_tar, sim2D, 0.))

100%|██████████| 20000/20000 [01:35<00:00, 208.48it/s]


In [14]:
df_scores = pd.DataFrame.from_records(scores)
print(df_scores.shape)
df_scores.head()

(20000, 4)


,0,1,2,3
0,N#CC1=CC=CC=C1COC1=CC=CC(C(=O)N2CCN(C3=CC=C(Br...,N#CC1=CC=CC(C(=O)N2CCN(CCCCN3CCN(C4=CC=CC=C4)C...,0.402778,0.984454
1,N#CC1=CC=CC=C1COC1=CC=CC(C(=O)N2CCN(C3=CC=C(Br...,N#CC1=CC=CC(C(=O)N2CCN(C3=CC=CC=C3)CC2)=C1,0.409091,0.000791
2,N#CC1=CC=CC=C1COC1=CC=CC(C(=O)N2CCN(C3=CC=C(Br...,N#CC1=CC=CC(C(=O)N2CCN(CCCCN3CCN(C4=CC=CC=C4)C...,0.402778,0.984454
3,N#CC1=CC=CC=C1COC1=CC=CC(C(=O)N2CCN(C3=CC=C(Br...,N#CC1=CC=CC(C(=O)N2CCN(CCCCN3CCN(C4=CC=CC=C4)C...,0.402778,0.984454
4,N#CC1=CC=CC=C1COC1=CC=CC(C(=O)N2CCN(C3=CC=C(Br...,N#CC1=CC=CC(C(=O)N2CCN(CCCCN3CCN(C4=CC=CC=C4F)...,0.381579,0.992198


## 9. Save the results

In [15]:
df_scores.to_csv(filepath_output, header=None, index=False)